# Generation: Generating a Response

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [3]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = OpenAIEmbeddings(model='text-embedding-ada-002'))

C:\Users\asus\AppData\Local\Temp\ipykernel_11096\4174848054.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures",


In [4]:
len(vectorstore.get()['documents'])

21

In [5]:
retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [6]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [7]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  max_tokens = 250)

C:\Users\asus\miniconda3\envs\langchain_env\lib\site-packages\IPython\core\interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [8]:
question = "What software do data scientists use?"

In [9]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template 
         | chat 
         | StrOutputParser())

In [10]:
chain.invoke(question)

"Data scientists use a variety of software and programming languages. R and Python are two of the most popular tools due to their ability to manipulate data and their integration within multiple data and data science software platforms. They can also solve a wide range of business and data-related problems. In addition, Hadoop, a software framework designed to handle the complexity and computational intensity of big data, is widely used. There are also several software designed for business intelligence visualizations, including Power BI, SaS, Qlik, and Tableau.\n\nResources: 'Programming Languages & Software Employed in Data Science - All the Tools You Need'"

In [12]:
print("Data scientists use a variety of software and programming languages. R and Python are two of the most popular tools due to their ability to manipulate data and their integration within multiple data and data science software platforms. They can also solve a wide range of business and data-related problems. In addition, Hadoop, a software framework designed to handle the complexity and computational intensity of big data, is widely used. There are also several software designed for business intelligence visualizations, including Power BI, SaS, Qlik, and Tableau.\n\nResources: 'Programming Languages & Software Employed in Data Science - All the Tools You Need")

Data scientists use a variety of software and programming languages. R and Python are two of the most popular tools due to their ability to manipulate data and their integration within multiple data and data science software platforms. They can also solve a wide range of business and data-related problems. In addition, Hadoop, a software framework designed to handle the complexity and computational intensity of big data, is widely used. There are also several software designed for business intelligence visualizations, including Power BI, SaS, Qlik, and Tableau.

Resources: 'Programming Languages & Software Employed in Data Science - All the Tools You Need
